In [1]:
%reload_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

In [2]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

gui = MetalGUI(design)

In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um',
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

gui.rebuild()
gui.autoscale()

In [4]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

In [5]:
hfss = eig_qb.sim.renderer

In [6]:
hfss.start()

INFO 06:45PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:45PM [load_ansys_project]: 	Opened Ansys App
INFO 06:45PM [load_ansys_project]: 	Opened Ansys Desktop v15.0.0
INFO 06:45PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/leona/Documents/Ansoft/
	Project:   Project1
INFO 06:45PM [connect_design]: No active design found (or error getting active design).
INFO 06:45PM [connect]: 	 Connected to project "Project1". No design detected


True

In [7]:
hfss.activate_ansys_design("TransmonQubit", 'eigenmode')  # use new_ansys_design() to force creation of a blank design

06:45PM 33s WARNING [activate_ansys_design]: The design_name=TransmonQubit was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 06:45PM [connect_design]: 	Opened active design
	Design:    TransmonQubit [Solution type: Eigenmode]
WARNING 06:45PM [connect_setup]: 	No design setup detected.
WARNING 06:45PM [connect_setup]: 	Creating eigenmode default setup.
INFO 06:45PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [8]:
hfss.render_design(['Q1'], [])
#hfss.save_screenshot()

In [10]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')

setup.analyze()

INFO 06:47PM [analyze]: Analyzing setup Setup



Number of eigenmodes to find             = 1
Number of simulation passes              = 10
Convergence freq max delta percent diff  = 0.1



In [12]:
eig_qb.sim.convergence_t, eig_qb.sim.convergence_f, _ = hfss.get_convergences()
eig_qb.sim.plot_convergences()

06:11PM 28s INFO [get_f_convergence]: Saved convergences to C:\Users\Leonardo Bove\Desktop\OneDrive - University of Pisa\Tesi\qiskit\qmetal\hfss_eig_f_convergence.csv


In [14]:
hfss.modeler._modeler.ShowWindow()
#hfss.plot_fields('main')
#hfss.save_screenshot()

In [15]:
hfss.plot_ansys_delete(['Mag_E1'])

06:19PM 37s WARNING [plot_ansys_delete]: This method is deprecated. Change your scripts to use clear_fields()


In [18]:
pinfo = hfss.pinfo
pinfo.junctions['jj'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj',
                             'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk'] = ['main'] # Dissipative elements: specify

In [19]:
eprd = epr.DistributedAnalysis(pinfo)

Design "TransmonQubit" info:
	# eigenmodes    1
	# variations    1


In [20]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag    = {ℰ_mag}
""")


ℰ_elec_all       = 2.62819345245582e-24
ℰ_elec_substrate = 2.41970903755386e-24
EPR of substrate = 92.1%

ℰ_mag    = 1.36442639274371e-26



In [21]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)

# 4b. Report solved results
swp_variable = 'Lj' # suppose we swept an optimetric analysis vs. inductance Lj_alice
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

  options=pd.Series(get_instance_vars(self.options)),




Variation 0  [1/1]


  Ljs = pd.Series({})

  Cjs = pd.Series({})



com_error: (-2147352567, 'Eccezione.', (0, None, None, None, 0, -2147024381), None)

In [22]:
eig_qb.sim.close()

Warning! 9 COM references still alive
Ansys will likely refuse to shut down
